In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as TF

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [7]:
# mushroom
from mushroom.mushroom import Mushroom
import mushroom.data.xenium as xenium
import mushroom.data.visium as visium
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils

/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [9]:
dtype_to_chkpt = {
    'xenium': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/xenium_HT413C1_Th1k4A1_v1/outputs/chkpts/last-v1.ckpt',
    'he': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/he_HT413C1_Th1k4A1_v1/outputs/chkpts/last.ckpt',
    'multiplex': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/multiplex_HT413C1_Th1k4A1_v1/outputs/chkpts/last.ckpt'
}
dtype_to_config = {
    'xenium': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/xenium_HT413C1_Th1k4A1_v1/outputs/chkpts/mushroom_config.yaml',
    'he': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/he_HT413C1_Th1k4A1_v1/outputs/chkpts/mushroom_config.yaml',
    'multiplex': '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/multiplex_HT413C1_Th1k4A1_v1/outputs/chkpts/mushroom_config.yaml',
}

In [10]:
dtype_to_mushroom = {k:Mushroom.from_config(
    dtype_to_config[k],
    chkpt_filepath=dtype_to_chkpt[k],
    accelerator='cpu'
) for k in dtype_to_chkpt.keys()}

INFO:root:starting xenium processing
INFO:root:using 477 channels
INFO:root:5 sections detected: ['s2', 's5', 's9', 's13', 's21']
INFO:root:processing sections
INFO:root:generating image data for section s2
INFO:root:generating image data for section s5
INFO:root:generating image data for section s9
INFO:root:generating image data for section s13
INFO:root:generating image data for section s21
INFO:root:generating training dataset
INFO:root:generating inference dataset
INFO:root:total of 5 sections detected: [('s2', 'xenium'), ('s5', 'xenium'), ('s9', 'xenium'), ('s13', 'xenium'), ('s21', 'xenium')]
INFO:root:creating data loaders
INFO:root:creating ViT
INFO:root:model initialized
wandb: Currently logged in as: estorrs (tme-st). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
INFO:root:loading checkpoint: /data/estorrs/mushroom/data/projects/htan_talk/mushroom/xenium_HT413C1_Th1k4A1_v1/outputs/chkpts/last-v1.ckpt


RuntimeError: Error(s) in loading state_dict for LitMushroom:
	Missing key(s) in state_dict: "sae.codes_to_integrated_logits.0.0.weight", "sae.codes_to_integrated_logits.0.0.bias", "sae.codes_to_integrated_logits.0.2.weight", "sae.codes_to_integrated_logits.0.2.bias", "sae.codes_to_integrated_logits.1.0.weight", "sae.codes_to_integrated_logits.1.0.bias", "sae.codes_to_integrated_logits.1.2.weight", "sae.codes_to_integrated_logits.1.2.bias", "sae.codes_to_integrated_logits.2.0.weight", "sae.codes_to_integrated_logits.2.0.bias", "sae.codes_to_integrated_logits.2.2.weight", "sae.codes_to_integrated_logits.2.2.bias", "sae.integrated_codebooks.0", "sae.integrated_codebooks.1", "sae.integrated_codebooks.2". 

In [ ]:
for dtype, mushroom in dtype_to_mushroom.items():
    mushroom.embed_sections()

In [5]:
he_volume = np.load(
    '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/xenium_HT413C1_Th1k4A1_v1/outputs/chkpts/outputs.npy',
    allow_pickle=True
).flat[0]
multiplex_volume = np.load(
    '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/multiplex_HT413C1_Th1k4A1_v1/outputs/chkpts/outputs.npy',
    allow_pickle=True
).flat[0]
visium_volume = np.load(
    '/data/estorrs/mushroom/data/projects/htan_talk/mushroom/xenium_HT413C1_Th1k4A1_v1/outputs/chkpts/outputs.npy',
    allow_pickle=True
).flat[0]
he_volume.shape, multiplex_volume.shape, visium_volume.shape

AttributeError: 'dict' object has no attribute 'shape'

In [6]:
he_volume.keys()

dict_keys(['cluster_ids', 'cluster_probs', 'cluster_volume'])